# GAN-BERT (in Pytorch and compatible with HuggingFace)

This is a Pytorch (+ **Huggingface** transformers) implementation of the GAN-BERT model from https://github.com/crux82/ganbert. While the original GAN-BERT was an extension of BERT, this implementation can be adapted to several architectures, ranging from Roberta to Albert!

**NOTE**: given that this implementation is different from the original one in Tensorflow, some results can be slighty different.


Let's GO!

Required Imports.

In [1]:
import time

start_time = time.time()
!pip install transformers==4.3.2
import torch
import io
import torch.nn.functional as F
import random
import numpy as np
import time
import math
import datetime
import torch.nn as nn
from transformers import *
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
#!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 -f https://download.pytorch.org/whl/torch_stable.html
#!pip install sentencepiece

##Set random values
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
if torch.cuda.is_available():
  torch.cuda.manual_seed_all(seed_val)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 13.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 49.7 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (tokenizers)


GroupViT models are not usable since `tensorflow_probability` can't be loaded. It seems you have `tensorflow_probability` installed with the wrong tensorflow version.Please try to reinstall it following the instructions here: https://github.com/tensorflow/probability.
TAPAS models are not usable since `tensorflow_probability` can't be loaded. It seems you have `tensorflow_probability` installed with the wrong tensorflow version. Please try to reinstall it following the instructions here: https://github.com/tensorflow/probability.


In [2]:
# If there's a GPU available...
if torch.cuda.is_available():
    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA A100-SXM4-40GB


### Input Parameters


In [3]:
#--------------------------------
#  Transformer parameters
#--------------------------------
max_seq_length = 64
batch_size = 64

#--------------------------------
#  GAN-BERT specific parameters
#--------------------------------
# number of hidden layers in the generator,
# each of the size of the output space
num_hidden_layers_g = 1;
# number of hidden layers in the discriminator,
# each of the size of the input space
num_hidden_layers_d = 1;
# size of the generator's input noisy vectors
noise_size = 100
# dropout to be applied to discriminator's input vectors
out_dropout_rate = 0.2

# Replicate labeled data to balance poorly represented datasets,
# e.g., less than 1% of labeled material
apply_balance = True

#--------------------------------
#  Optimization parameters
#--------------------------------
learning_rate_discriminator = 5e-5
learning_rate_generator = 5e-5
epsilon = 1e-8
num_train_epochs = 30
multi_gpu = True
# Scheduler
apply_scheduler = False
warmup_proportion = 0.1
# Print
print_each_n_step = 10

#--------------------------------
#  Adopted Tranformer model
#--------------------------------
# Since this version is compatible with Huggingface transformers, you can uncomment
# (or add) transformer models compatible with GAN

model_name = "bert-base-cased"
#model_name = "bert-base-uncased"
#model_name = "roberta-base"
#model_name = "albert-base-v2"
#model_name = "xlm-roberta-base"
#model_name = "amazon/bort"

#--------------------------------
#  Retrieve the TREC QC Dataset
#--------------------------------
! git clone https://github.com/crux82/ganbert

#  NOTE: in this setting 50 classes are involved
labeled_file = "./ganbert/data/labeled.tsv"
unlabeled_file = "./ganbert/data/unlabeled.tsv"
test_filename = "./ganbert/data/test.tsv"

label_list = ["UNK_UNK","ABBR_abb", "ABBR_exp", "DESC_def", "DESC_desc",
              "DESC_manner", "DESC_reason", "ENTY_animal", "ENTY_body",
              "ENTY_color", "ENTY_cremat", "ENTY_currency", "ENTY_dismed",
              "ENTY_event", "ENTY_food", "ENTY_instru", "ENTY_lang",
              "ENTY_letter", "ENTY_other", "ENTY_plant", "ENTY_product",
              "ENTY_religion", "ENTY_sport", "ENTY_substance", "ENTY_symbol",
              "ENTY_techmeth", "ENTY_termeq", "ENTY_veh", "ENTY_word", "HUM_desc",
              "HUM_gr", "HUM_ind", "HUM_title", "LOC_city", "LOC_country",
              "LOC_mount", "LOC_other", "LOC_state", "NUM_code", "NUM_count",
              "NUM_date", "NUM_dist", "NUM_money", "NUM_ord", "NUM_other",
              "NUM_perc", "NUM_period", "NUM_speed", "NUM_temp", "NUM_volsize",
              "NUM_weight"]

Cloning into 'ganbert'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 77 (delta 32), reused 25 (delta 25), pack-reused 39 (from 1)
Receiving objects: 100% (77/77), 678.10 KiB | 25.11 MiB/s, done.
Resolving deltas: 100% (33/33), done.


Load the Tranformer Model

In [4]:
transformer = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/cd5ef92a9fb2f889e972770a36d4ed042daf221e/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.47.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}



model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/cd5ef92a9fb2f889e972770a36d4ed042daf221e/model.safetensors
A pretrained model of type `BertModel` contains parameters that have been renamed internally (a few are listed below but more are present in the model):
* `bert.embeddings.LayerNorm.gamma` -> `bert.embeddings.LayerNorm.weight`
* `bert.encoder.layer.0.attention.output.LayerNorm.gamma` -> `{'bert.embeddings.LayerNorm.gamma': 'bert.embeddings.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.gamma': {...}, 'bert.encoder.layer.0.output.LayerNorm.gamma': {...}, 'bert.encoder.layer.1.attention.output.LayerNorm.gamma': {...}, 'bert.encoder.layer.1.output.LayerNorm.gamma': {...}, 'bert.encoder.layer.10.attention.output.LayerNorm.gamma': {...}, 'bert.encoder.layer.10.output.LayerNorm.gamma': {...}, 'bert.encoder.layer.11.attention.output.LayerNorm.gamma': {...}, 'bert.encoder.layer.11.output.LayerNorm.gam

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/cd5ef92a9fb2f889e972770a36d4ed042daf221e/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.47.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}



vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/cd5ef92a9fb2f889e972770a36d4ed042daf221e/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/cd5ef92a9fb2f889e972770a36d4ed042daf221e/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/cd5ef92a9fb2f889e972770a36d4ed042daf221e/tokenizer_config.json
loading file chat_template.jinja from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/cd5ef92a9fb2f889e972770a36d4ed042daf221e/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gr

Function required to load the dataset

In [5]:
def get_qc_examples(input_file):
  """Creates examples for the training and dev sets."""
  examples = []

  with open(input_file, 'r') as f:
      contents = f.read()
      file_as_list = contents.splitlines()
      for line in file_as_list[1:]:
          split = line.split(" ")
          question = ' '.join(split[1:])

          text_a = question
          inn_split = split[0].split(":")
          label = inn_split[0] + "_" + inn_split[1]
          examples.append((text_a, label))
      f.close()

  return examples

**Load** the input QC dataset (fine-grained)

In [6]:
#Load the examples
labeled_examples = get_qc_examples(labeled_file)
#unlabeled_examples = get_qc_examples(unlabeled_file)
#test_examples = get_qc_examples(test_filename)

Functions required to convert examples into Dataloader

In [7]:
def generate_data_loader(input_examples, label_masks, label_map, do_shuffle = False, balance_label_examples = False):
  '''
  Generate a Dataloader given the input examples, eventually masked if they are
  to be considered NOT labeled.
  '''
  examples = []

  # Count the percentage of labeled examples
  num_labeled_examples = 0
  for label_mask in label_masks:
    if label_mask:
      num_labeled_examples += 1
  label_mask_rate = num_labeled_examples/len(input_examples)

  # if required it applies the balance
  for index, ex in enumerate(input_examples):
    if label_mask_rate == 1 or not balance_label_examples:
      examples.append((ex, label_masks[index]))
    else:
      # IT SIMULATE A LABELED EXAMPLE
      if label_masks[index]:
        balance = int(1/label_mask_rate)
        balance = int(math.log(balance,2))
        if balance < 1:
          balance = 1
        for b in range(0, int(balance)):
          examples.append((ex, label_masks[index]))
      else:
        examples.append((ex, label_masks[index]))

  #-----------------------------------------------
  # Generate input examples to the Transformer
  #-----------------------------------------------
  input_ids = []
  input_mask_array = []
  label_mask_array = []
  label_id_array = []

  # Tokenization
  for (text, label_mask) in examples:
    encoded_sent = tokenizer.encode(text[0], add_special_tokens=True, max_length=max_seq_length, padding="max_length", truncation=True)
    input_ids.append(encoded_sent)
    label_id_array.append(label_map[text[1]])
    label_mask_array.append(label_mask)

  # Attention to token (to ignore padded input wordpieces)
  for sent in input_ids:
    att_mask = [int(token_id > 0) for token_id in sent]
    input_mask_array.append(att_mask)
  # Convertion to Tensor
  input_ids = torch.tensor(input_ids)
  input_mask_array = torch.tensor(input_mask_array)
  label_id_array = torch.tensor(label_id_array, dtype=torch.long)
  label_mask_array = torch.tensor(label_mask_array)

  # Building the TensorDataset
  dataset = TensorDataset(input_ids, input_mask_array, label_id_array, label_mask_array)

  if do_shuffle:
    sampler = RandomSampler
  else:
    sampler = SequentialSampler

  # Building the DataLoader
  return DataLoader(
              dataset,  # The training samples.
              sampler = sampler(dataset),
              batch_size = batch_size) # Trains with this batch size.

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

Convert the input examples into DataLoader

In [8]:
# Criar o mapeamento de labels
label_map = {}
for (i, label) in enumerate(label_list):
    label_map[label] = i

# ------------------------------
#   Carregar o conjunto de treinamento
# ------------------------------
train_examples = labeled_examples

# Normalizar os rótulos em train_examples
train_examples = [(text[0], ' '.join(text[1].split()).replace("\t", "_")) for text in train_examples]

# Atualizar label_map dinamicamente
dataset_labels = set(example[1] for example in train_examples)
for label in dataset_labels:
    if label not in label_map:
        label_map[label] = len(label_map)

print("Updated label_map:", label_map)

# Gerar a máscara para os rótulos de treinamento
train_label_masks = np.ones(len(train_examples), dtype=bool)

# Gerar o dataloader novamente
train_dataloader = generate_data_loader(
    train_examples,
    train_label_masks,
    label_map,
    do_shuffle=True,
    balance_label_examples=apply_balance
)

# ------------------------------
#   Carregar o conjunto de teste (opcional)
# ------------------------------
# O dataset de teste (se necessário) pode ser processado de forma similar
# test_label_masks = np.ones(len(test_examples), dtype=bool)
# test_dataloader = generate_data_loader(
#     test_examples,
#     test_label_masks,
#     label_map,
#     do_shuffle=False,
#     balance_label_examples=False
# )


Updated label_map: {'UNK_UNK': 0, 'ABBR_abb': 1, 'ABBR_exp': 2, 'DESC_def': 3, 'DESC_desc': 4, 'DESC_manner': 5, 'DESC_reason': 6, 'ENTY_animal': 7, 'ENTY_body': 8, 'ENTY_color': 9, 'ENTY_cremat': 10, 'ENTY_currency': 11, 'ENTY_dismed': 12, 'ENTY_event': 13, 'ENTY_food': 14, 'ENTY_instru': 15, 'ENTY_lang': 16, 'ENTY_letter': 17, 'ENTY_other': 18, 'ENTY_plant': 19, 'ENTY_product': 20, 'ENTY_religion': 21, 'ENTY_sport': 22, 'ENTY_substance': 23, 'ENTY_symbol': 24, 'ENTY_techmeth': 25, 'ENTY_termeq': 26, 'ENTY_veh': 27, 'ENTY_word': 28, 'HUM_desc': 29, 'HUM_gr': 30, 'HUM_ind': 31, 'HUM_title': 32, 'LOC_city': 33, 'LOC_country': 34, 'LOC_mount': 35, 'LOC_other': 36, 'LOC_state': 37, 'NUM_code': 38, 'NUM_count': 39, 'NUM_date': 40, 'NUM_dist': 41, 'NUM_money': 42, 'NUM_ord': 43, 'NUM_other': 44, 'NUM_perc': 45, 'NUM_period': 46, 'NUM_speed': 47, 'NUM_temp': 48, 'NUM_volsize': 49, 'NUM_weight': 50}


<ipython-input-7-16c38d5f7491>:54: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  label_mask_array = torch.tensor(label_mask_array)


We define the Generator and Discriminator as discussed in https://www.aclweb.org/anthology/2020.acl-main.191/

In [9]:
'''#------------------------------
#   The Generator as in
#   https://www.aclweb.org/anthology/2020.acl-main.191/
#   https://github.com/crux82/ganbert
#------------------------------
class Generator(nn.Module):
    def __init__(self, noise_size=100, output_size=512, hidden_sizes=[512], dropout_rate=0.1):
        super(Generator, self).__init__()
        layers = []
        hidden_sizes = [noise_size] + hidden_sizes
        for i in range(len(hidden_sizes)-1):
            layers.extend([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]), nn.LeakyReLU(0.2, inplace=True), nn.Dropout(dropout_rate)])

        layers.append(nn.Linear(hidden_sizes[-1],output_size))
        self.layers = nn.Sequential(*layers)

    def forward(self, noise):
        output_rep = self.layers(noise)
        return output_rep

#------------------------------
#   The Discriminator
#   https://www.aclweb.org/anthology/2020.acl-main.191/
#   https://github.com/crux82/ganbert
#------------------------------
class Discriminator(nn.Module):
    def __init__(self, input_size=512, hidden_sizes=[512], num_labels=2, dropout_rate=0.1):
        super(Discriminator, self).__init__()
        self.input_dropout = nn.Dropout(p=dropout_rate)
        layers = []
        hidden_sizes = [input_size] + hidden_sizes
        for i in range(len(hidden_sizes)-1):
            layers.extend([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]), nn.LeakyReLU(0.2, inplace=True), nn.Dropout(dropout_rate)])

        self.layers = nn.Sequential(*layers) #per il flatten
        self.logit = nn.Linear(hidden_sizes[-1],num_labels+1) # +1 for the probability of this sample being fake/real.
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, input_rep):
        input_rep = self.input_dropout(input_rep)
        last_rep = self.layers(input_rep)
        logits = self.logit(last_rep)
        probs = self.softmax(logits)
        return last_rep, logits, probs'''

'#------------------------------\n#   The Generator as in\n#   https://www.aclweb.org/anthology/2020.acl-main.191/\n#   https://github.com/crux82/ganbert\n#------------------------------\nclass Generator(nn.Module):\n    def __init__(self, noise_size=100, output_size=512, hidden_sizes=[512], dropout_rate=0.1):\n        super(Generator, self).__init__()\n        layers = []\n        hidden_sizes = [noise_size] + hidden_sizes\n        for i in range(len(hidden_sizes)-1):\n            layers.extend([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]), nn.LeakyReLU(0.2, inplace=True), nn.Dropout(dropout_rate)])\n\n        layers.append(nn.Linear(hidden_sizes[-1],output_size))\n        self.layers = nn.Sequential(*layers)\n\n    def forward(self, noise):\n        output_rep = self.layers(noise)\n        return output_rep\n\n#------------------------------\n#   The Discriminator\n#   https://www.aclweb.org/anthology/2020.acl-main.191/\n#   https://github.com/crux82/ganbert\n#----------------------

We instantiate the Discriminator and Generator

In [10]:
'''# The config file is required to get the dimension of the vector produced by
# the underlying transformer
config = AutoConfig.from_pretrained(model_name)
hidden_size = int(config.hidden_size)
# Define the number and width of hidden layers
hidden_levels_g = [hidden_size for i in range(0, num_hidden_layers_g)]
hidden_levels_d = [hidden_size for i in range(0, num_hidden_layers_d)]

#-------------------------------------------------
#   Instantiate the Generator and Discriminator
#-------------------------------------------------
generator = Generator(noise_size=noise_size, output_size=hidden_size, hidden_sizes=hidden_levels_g, dropout_rate=out_dropout_rate)
discriminator = Discriminator(input_size=hidden_size, hidden_sizes=hidden_levels_d, num_labels=len(label_list), dropout_rate=out_dropout_rate)

# Put everything in the GPU if available
if torch.cuda.is_available():
  generator.cuda()
  discriminator.cuda()
  transformer.cuda()
  if multi_gpu:
    transformer = torch.nn.DataParallel(transformer)

# print(config)'''

'# The config file is required to get the dimension of the vector produced by\n# the underlying transformer\nconfig = AutoConfig.from_pretrained(model_name)\nhidden_size = int(config.hidden_size)\n# Define the number and width of hidden layers\nhidden_levels_g = [hidden_size for i in range(0, num_hidden_layers_g)]\nhidden_levels_d = [hidden_size for i in range(0, num_hidden_layers_d)]\n\n#-------------------------------------------------\n#   Instantiate the Generator and Discriminator\n#-------------------------------------------------\ngenerator = Generator(noise_size=noise_size, output_size=hidden_size, hidden_sizes=hidden_levels_g, dropout_rate=out_dropout_rate)\ndiscriminator = Discriminator(input_size=hidden_size, hidden_sizes=hidden_levels_d, num_labels=len(label_list), dropout_rate=out_dropout_rate)\n\n# Put everything in the GPU if available\nif torch.cuda.is_available():\n  generator.cuda()\n  discriminator.cuda()\n  transformer.cuda()\n  if multi_gpu:\n    transformer = torc

Let's go with the training procedure

In [11]:
import time
import torch
import numpy as np
from transformers import *

# Configuração do dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Inicialização do modelo
transformer = transformer.to(device)
transformer.train()

# Otimizador
optimizer = torch.optim.AdamW(transformer.parameters(), lr=5e-5)

# Função para calcular métricas
def calculate_metrics(y_true, y_pred):
    # Classes únicas
    classes = np.unique(y_true)

    recalls = []
    precisions = []
    f1_scores = []

    for cls in classes:
        # True Positives, False Positives, False Negatives
        tp = np.sum((y_true == cls) & (y_pred == cls))
        fp = np.sum((y_true != cls) & (y_pred == cls))
        fn = np.sum((y_true == cls) & (y_pred != cls))

        # Precision
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0

        # Recall
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0

        # F1 Score
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1_score)

    # Média das métricas por classe
    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)
    avg_f1 = np.mean(f1_scores)

    return avg_precision, avg_recall, avg_f1

# Loop de treinamento
training_stats = []
total_t0 = time.time()

# Inicializar listas para previsões e rótulos finais
final_preds = []
final_labels = []

for epoch_i in range(0, num_train_epochs):
    print(f"======== Epoch {epoch_i + 1} / {num_train_epochs} ========")
    print("Training...")

    t0 = time.time()
    tr_loss = 0
    correct_predictions = 0
    total_predictions = 0

    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        optimizer.zero_grad()

        outputs = transformer(b_input_ids, attention_mask=b_input_mask)
        logits = outputs.last_hidden_state[:, 0, :]
        loss_fn = torch.nn.CrossEntropyLoss()
        loss = loss_fn(logits, b_labels)

        loss.backward()
        optimizer.step()

        tr_loss += loss.item()

        # Cálculo da acurácia
        preds = torch.argmax(logits, dim=1)
        correct_predictions += (preds == b_labels).sum().item()
        total_predictions += b_labels.size(0)

        # Armazenar previsões e rótulos finais
        final_preds.extend(preds.detach().cpu())
        final_labels.extend(b_labels.detach().cpu())

    # Cálculo do loss médio e acurácia
    avg_train_loss = tr_loss / len(train_dataloader)
    train_accuracy = correct_predictions / total_predictions

    print(f"  Training Accuracy: {train_accuracy:.3f}")
    print(f"  Average Training Loss: {avg_train_loss:.3f}")

    # Salvar estatísticas
    training_stats.append({
        'epoch': epoch_i + 1,
        'Training Loss': avg_train_loss,
        'Training Accuracy': train_accuracy,
    })

# Avaliação final com cálculo de métricas
print("\nEvaluating the model...")

# Conversão para numpy
final_preds = torch.stack(final_preds).numpy()
final_labels = torch.stack(final_labels).numpy()

# Verificar valores únicos em rótulos e predições
print("Unique labels in final_labels:", np.unique(final_labels))
print("Unique predictions in final_preds:", np.unique(final_preds))

# Cálculo das métricas
precision, recall, f1_score = calculate_metrics(final_labels, final_preds)

print("\nFinal Metrics:")
print(f"  Precision: {precision:.3f}")
print(f"  Recall: {recall:.3f}")
print(f"  F1 Score: {f1_score:.3f}")

# Salvando o modelo treinado
model_save_path = "bert_model.pth"
torch.save(transformer.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

# Exibindo estatísticas finais
print("\nTraining complete.")
for stat in training_stats:
    print(f"Epoch {stat['epoch']}: Loss={stat['Training Loss']:.3f}, "
          f"Accuracy={stat['Training Accuracy']:.3f}")

print(f"\nFinal Precision: {precision:.3f}, Recall: {recall:.3f}, F1 Score: {f1_score:.3f}")


Using device: cuda
======== Epoch 1 / 30 ========
Training...
  Training Accuracy: 0.000
  Average Training Loss: 7.312
======== Epoch 2 / 30 ========
Training...
  Training Accuracy: 0.064
  Average Training Loss: 6.344
======== Epoch 3 / 30 ========
Training...
  Training Accuracy: 0.202
  Average Training Loss: 5.924
======== Epoch 4 / 30 ========
Training...
  Training Accuracy: 0.248
  Average Training Loss: 5.491
======== Epoch 5 / 30 ========
Training...
  Training Accuracy: 0.257
  Average Training Loss: 4.725
======== Epoch 6 / 30 ========
Training...
  Training Accuracy: 0.248
  Average Training Loss: 4.087
======== Epoch 7 / 30 ========
Training...
  Training Accuracy: 0.349
  Average Training Loss: 3.514
======== Epoch 8 / 30 ========
Training...
  Training Accuracy: 0.385
  Average Training Loss: 3.123
======== Epoch 9 / 30 ========
Training...
  Training Accuracy: 0.440
  Average Training Loss: 2.636
======== Epoch 10 / 30 ========
Training...
  Training Accuracy: 0.523
 

In [12]:
for stat in training_stats:
  print(stat)

print("\nTraining complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

resultado = sum(range(10**6))

end_time = time.time()
execution_time = end_time - start_time
print(f"Tempo de execução: {execution_time:.4f} segundos")

{'epoch': 1, 'Training Loss': 7.311760902404785, 'Training Accuracy': 0.0}
{'epoch': 2, 'Training Loss': 6.343673467636108, 'Training Accuracy': 0.06422018348623854}
{'epoch': 3, 'Training Loss': 5.924002408981323, 'Training Accuracy': 0.2018348623853211}
{'epoch': 4, 'Training Loss': 5.4913489818573, 'Training Accuracy': 0.24770642201834864}
{'epoch': 5, 'Training Loss': 4.724750518798828, 'Training Accuracy': 0.25688073394495414}
{'epoch': 6, 'Training Loss': 4.087282419204712, 'Training Accuracy': 0.24770642201834864}
{'epoch': 7, 'Training Loss': 3.514300227165222, 'Training Accuracy': 0.3486238532110092}
{'epoch': 8, 'Training Loss': 3.1226491928100586, 'Training Accuracy': 0.3853211009174312}
{'epoch': 9, 'Training Loss': 2.6355944871902466, 'Training Accuracy': 0.44036697247706424}
{'epoch': 10, 'Training Loss': 2.2759865522384644, 'Training Accuracy': 0.5229357798165137}
{'epoch': 11, 'Training Loss': 1.7886714935302734, 'Training Accuracy': 0.6605504587155964}
{'epoch': 12, 'T

In [13]:
# Salvar o modelo transformer
torch.save(transformer.state_dict(), 'transformer.pth')
print("Transformer model saved.")

# Carregar o modelo transformer
transformer.load_state_dict(torch.load('transformer.pth'))
transformer.eval()
print("Transformer model loaded and set to evaluation mode.")


Transformer model saved.


<ipython-input-13-c35a707ee44d>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  transformer.load_state_dict(torch.load('transformer.pth'))


Transformer model loaded and set to evaluation mode.


In [14]:
def preprocess(text, max_seq_length=64):
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    encoded_sent = tokenizer.encode(
        text,
        add_special_tokens=True,
        max_length=max_seq_length,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )
    attention_mask = (encoded_sent > 0).long()
    return encoded_sent, attention_mask


In [15]:
def classify_question(text, label_list):
    transformer.eval()

    # Pré-processar o texto
    input_ids, attention_mask = preprocess(text)
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)

    with torch.no_grad():
        # Obter os logits do transformer
        outputs = transformer(input_ids, attention_mask=attention_mask)
        logits = outputs.last_hidden_state[:, 0, :]  # Usar representação do token [CLS]

        # Predição
        pred = torch.argmax(logits, dim=1).item()

    return label_list[pred]
